# Demo and test environments

In [1]:
%reload_ext autoreload
%autoreload 2
import gym
import matplotlib.pyplot as plt
from ezai.env import env_info
import numpy as np
import copy
import yaml

In [2]:
from IPython.display import display,Javascript
import ipywidgets
env_map={'ride-v0':'0.4.3',
        'arora-v0':'2.10.15'}
env_dd = ipywidgets.Dropdown(
       options=list(env_map.keys()),
       value=list(env_map.keys())[0],
       description='Environment:')
ver = ipywidgets.Text(
       value=list(env_map.values())[0],
       description='Version:', )
def nxtHandler(b):
    display(Javascript('Jupyter.notebook.select_next().get_selected_cell()'))
    #get_ipython().set_next_input(replace=False)
    
nxt = ipywidgets.Button(description='Next Cell')
nxt.on_click(nxtHandler)

def update_ver(*args):
    ver.value = env_map.get(env_dd.value)
    
env_dd.observe(update_ver, 'value')
box = ipywidgets.HBox([env_dd,ver,nxt])
display(box)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
env_id = env_dd.value

if env_id=="ride-v0":
#from navsim_envs import ride as navenv
    from navsim_envs.ride import default_env_config
    from navsim_envs.ride import RideUnityEnv as UnityEnv
    from navsim_envs.ride import RideGymEnv as GymEnv
    env_path = f"/data/work/unity-envs/RIDE_{ver.value}/RIDE"
elif env_id=="arora-v0":
    from navsim_envs.arora import default_env_config
    from navsim_envs.arora import AroraUnityEnv as UnityEnv
    from navsim_envs.arora import AroraGymEnv as GymEnv
    env_path = f"/data/work/unity-envs/ARORA_{ver.value}/ARORA"
print(env_id,':',env_path)

arora-v0 : /data/work/unity-envs/ARORA_2.10.15/ARORA


In [23]:
env_config = None
def config():
    global env_config
    env_config = default_env_config.copy()
    env_config["env_path"]=env_path
    env_config["seed"]=123
    env_config["timeout"]=60
    env_config["obs_mode"]=1
    print(yaml.dump(env_config))
config()

agent_car_physics: 0
area: 0
base_port: 5004
debug: false
env_gpu_id: 0
env_path: /data/work/unity-envs/ARORA_2.10.15/ARORA
episode_max_steps: 1000
goal: 0
goal_clearance: 2.5
goal_distance: 50
log_folder: ./env_log
obs_height: 64
obs_mode: 1
obs_width: 64
relative_steering: true
reward_collision_mul: 4
reward_for_goal: 50
reward_for_no_viable_path: -50
reward_spl_delta_mul: 1
reward_step_mul: 0.1
save_actions: true
save_vector_obs: true
save_visual_obs: true
seed: 123
segmentation_mode: 1
show_visual: false
start_from_episode: 1
task: 0
timeout: 60
traffic_vehicles: 0
worker_id: 0



# The UnityEnv

In [24]:
config()

agent_car_physics: 0
area: 0
base_port: 5004
debug: false
env_gpu_id: 0
env_path: /data/work/unity-envs/ARORA_2.10.15/ARORA
episode_max_steps: 1000
goal: 0
goal_clearance: 2.5
goal_distance: 50
log_folder: ./env_log
obs_height: 64
obs_mode: 1
obs_width: 64
relative_steering: true
reward_collision_mul: 4
reward_for_goal: 50
reward_for_no_viable_path: -50
reward_spl_delta_mul: 1
reward_step_mul: 0.1
save_actions: true
save_vector_obs: true
save_visual_obs: true
seed: 123
segmentation_mode: 1
show_visual: false
start_from_episode: 1
task: 0
timeout: 60
traffic_vehicles: 0
worker_id: 0



In [25]:
#env_config["env_path"]=None
uenv = UnityEnv(env_config)

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
[INFO] Created UnityEnvironment from /mnt/disk1/work/unity-envs/ARORA_2.10.15/ARORA at port 5005 to start from episode 1


In [26]:
uenv.reset()

[INFO] Connected new brain: VectorNavigator?team=0
I am in reset
fpc: 3284.0 2666.0


In [27]:
print(uenv.unity_max_x,uenv.unity_max_z)

3284.0 2666.0


In [28]:
print(uenv.navmap_max_x,uenv.navmap_max_y)

3284 2666


In [29]:
print(uenv.shortest_path)

inside channel code 1: [2467.2001953125, 35.97684097290039, 788.800048828125, 2497.33349609375, 35.66666793823242, 812.0000610351562, 2498.33349609375, 35.66666793823242, 813.0000610351562, 2504.041015625, 35.77082824707031, 821.8750610351562]
inside channel code 2: [[2467.20019531   35.97684097  788.80004883]
 [2497.33349609   35.66666794  812.00006104]
 [2498.33349609   35.66666794  813.00006104]
 [2504.04101562   35.77082825  821.87506104]]
[[2467.20019531   35.97684097  788.80004883]
 [2497.33349609   35.66666794  812.00006104]
 [2498.33349609   35.66666794  813.00006104]
 [2504.04101562   35.77082825  821.87506104]]


In [30]:
uenv.close()

# The GymEnv

In [ ]:
# look at all observation modes
for i in UnityEnv.observation_modes:
    env_config["obs_mode"]=i
    env = gym.make(env_id, env_config=env_config) # ride.RideGymEnv(env_config)
    print('Observation Mode {}'.format(env_config["obs_mode"]))
    env_info(env)
    obs = env.reset()
    env.close()
    del env

In [ ]:
# test for 5 steps forward and 5 steps back 
env_config["obs_mode"]=1
env_config["env_gpu_id"]=0
#env_config["show_visual"]=True

env = gym.make(env_id, env_config=env_config)

steps = 5
obs = env.reset()
#print(obs)
print("Forward:")
for i in range(steps):
    obs = env.step(UnityEnv.actions['forward'])
    #print(obs[-1])
    #plt.imshow(env.render(mode='rgb_array'))
    #plt.show()
    #plt.imshow(env.render(mode='depth'))
    #plt.show()
    #plt.imshow(env.render(mode='segmentation'))
    #plt.show()
print("Backward:")
for i in range(steps):
    obs = env.step(UnityEnv.actions['backward'])
    #print(obs[-1])
    #plt.imshow(env.render(mode='rgb_array'))
    #plt.show()
    #plt.imshow(env.render(mode='depth'))
    #plt.show()
    #plt.imshow(env.render(mode='segmentation'))
    #plt.show()
env.close()
del env

In [ ]:
# test for 5 steps forward and 5 steps back 
env_config["obs_mode"]=1
env_config["env_gpu_id"]=1
#env_config["show_visual"]=True

env = gym.make(env_id, env_config=env_config)

steps = 200
obs = env.reset()
#print(obs)
print("Forward:")
for i in range(steps):
    obs = env.step(UnityEnv.actions['forward'])
    #print(obs[-1])
    #plt.imshow(env.render(mode='rgb_array'))
    #plt.show()
    #plt.imshow(env.render(mode='depth'))
    #plt.show()
    #plt.imshow(env.render(mode='segmentation'))
    #plt.show()
print("Backward:")
for i in range(steps):
    obs = env.step(UnityEnv.actions['backward'])
    #print(obs[-1])
    #plt.imshow(env.render(mode='rgb_array'))
    #plt.show()
    #plt.imshow(env.render(mode='depth'))
    #plt.show()
    #plt.imshow(env.render(mode='segmentation'))
    #plt.show()
env.close()
del env

# The mapping capabilities

In [ ]:
config()

In [ ]:
env = gym.make(env_id, env_config=env_config)

In [ ]:
navmap = env.get_navigable_map()

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(navmap,origin='lower',cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
area = env.get_navigable_map_zoom_area(0,0,29,19)
print(area.shape)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(area,origin='lower',cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(navmap[0:20,0:30],origin='lower',cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
env.close()

In [ ]:
env_config["env_gpu_id"]=1
uenv = UnityEnv(env_config)
uenv.reset()
uenv.close()
!nvidia-smi
!ps -aux | grep crt

# pending bugs

1. When you call ARORA shortest_path second time or more, the path is empty.
2. For RIDE, shortest path isnt even coming
3. 